In [9]:
from google.colab import drive
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import InceptionV3

In [10]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [11]:
drive.mount("/content/drive/", force_remount=True)

Mounted at /content/drive/


In [12]:
train_dir = '/content/drive/MyDrive/strawberry/sl_train'
validation_dir = '/content/drive/MyDrive/strawberry/sl_test'

In [13]:
IMG_SIZE = (224, 224)
BATCH_SIZE = 32

In [14]:
train_datagen = ImageDataGenerator(rescale=1./255,
                                   rotation_range=20,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True,
                                   vertical_flip=True)


In [15]:
validation_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(train_dir,
                                                    target_size=IMG_SIZE,
                                                    batch_size=BATCH_SIZE,
                                                    class_mode='categorical')

validation_generator = validation_datagen.flow_from_directory(validation_dir,
                                                              target_size=IMG_SIZE,
                                                              batch_size=BATCH_SIZE,
                                                              class_mode='categorical')

Found 338 images belonging to 2 classes.
Found 252 images belonging to 2 classes.


In [16]:
# Load the InceptionV3 model and freeze its layers
base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(IMG_SIZE[0], IMG_SIZE[1], 3))
for layer in base_model.layers:
    layer.trainable = False

87910968/87910968 [==============================] - 5s 0us/step


In [17]:
# Add a custom classifier on top of the base model
model = tf.keras.Sequential([
    base_model,
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(2, activation='softmax')
])

In [18]:
# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [19]:
# Train the model
history = model.fit(train_generator,
                    epochs=10,
                    validation_data=validation_generator)

Epoch 1/10
11/11 [==============================] - 104s 9s/step - loss: 0.3493 - accuracy: 0.8491 - val_loss: 0.2286 - val_accuracy: 0.9048
Epoch 2/10
11/11 [==============================] - 70s 7s/step - loss: 0.1103 - accuracy: 0.9586 - val_loss: 0.0913 - val_accuracy: 0.9762
Epoch 3/10
11/11 [==============================] - 83s 8s/step - loss: 0.0564 - accuracy: 0.9822 - val_loss: 0.1051 - val_accuracy: 0.9563
Epoch 4/10
11/11 [==============================] - 81s 8s/step - loss: 0.0663 - accuracy: 0.9852 - val_loss: 0.0664 - val_accuracy: 0.9802
Epoch 5/10
11/11 [==============================] - 81s 8s/step - loss: 0.0583 - accuracy: 0.9822 - val_loss: 0.0627 - val_accuracy: 0.9841
Epoch 6/10
11/11 [==============================] - 82s 8s/step - loss: 0.0320 - accuracy: 0.9941 - val_loss: 0.0664 - val_accuracy: 0.9762
Epoch 7/10
11/11 [==============================] - 81s 8s/step - loss: 0.0405 - accuracy: 0.9941 - val_loss: 0.0507 - val_accuracy: 0.9841
Epoch 8/10
11/11 [=

In [21]:
loss, accuracy = model.evaluate(validation_generator)
print('Validation accuracy:', accuracy)

8/8 [==============================] - 30s 3s/step - loss: 0.0452 - accuracy: 0.9881
Validation accuracy: 0.988095223903656
